In [1]:
import numpy as np
from scipy import linalg, optimize

https://stackoverflow.com/questions/78841904/python-scipy-optimize-gives-wrong-answer-how-to-deal-with-semidefinite-positive

```
It seems you have two key points to address:
  
  - Constraints must be somehow differentiable;
  - Initial guess must be different from `np.zeros((2, 2))`.

Let's adapt your MCVE:

    import numpy as np
    from scipy import linalg, optimize
    
    np.random.seed(123)
    
    def objective(x, C):
        A0 = x[:4].reshape((2, 2))  
        B0 = x[4:8].reshape((2, 2))
        A0 = A0 + A0.T
        B0 = B0 + B0.T
        D00 = np.kron(A0, B0)
        return - np.trace(D00 @ C)
    
    C = np.random.rand(4, 4)
    
We change initial guess for `np.ones((2, 2))`:

    A0_initial = np.ones((2,2))
    B0_initial = np.ones((2,2))
    initial_guess = np.hstack([A0_initial.ravel(), B0_initial.ravel()])

We rewrite the constraint function to be somehow more smooth and eventually differentiable:

    def constraint(x, sign=+1.):
        A0 = x.reshape((2, 2))
        A0 = A0 + A0.T
        U, s, Vt = linalg.svd(np.eye(2) + sign * A0)
        return np.min(np.sign(s)) * np.prod(np.abs(s))

Then your constraints can be translated as:

    constraints = [
        {'type': 'ineq', 'fun': lambda x: constraint(x[:4], sign=-1.)},
        {'type': 'ineq', 'fun': lambda x: constraint(x[:4], sign=+1.)},
        {'type': 'ineq', 'fun': lambda x: constraint(x[4:8], sign=-1.)},
        {'type': 'ineq', 'fun': lambda x: constraint(x[4:8], sign=+1.)},
    ]

And the optimization goes as follow:

    result = optimize.minimize(objective, initial_guess, args=(C,), constraints=constraints)
     # message: Optimization terminated successfully
     # success: True
     #  status: 0
     #     fun: -2.194729145491076e+17
     #       x: [ 8.354e+07  7.744e+07  7.744e+07  8.840e+07  6.991e+07
     #            8.913e+07  8.913e+07  7.693e+07]
     #     nit: 5
     #     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
     #            0.000e+00  0.000e+00  0.000e+00]
     #    nfev: 45
     #    njev: 5

Notice than both `A0` and `B0` are real symmetric matrices, therefore their eigenvalues are real as well.

To ensure positive definiteness, we also need to check constraints are fulfilled:

    constraint(result.x[:4], sign=-1.)  # 5555271365900154.0
    constraint(result.x[:4], sign=+1.)  # 5555271365900154.0
    constraint(result.x[4:8], sign=-1.) # 1.0262680544836974e+16
    constraint(result.x[4:8], sign=+1.) # 1.0262679957475332e+16

Which is the case in the current example, but it not guarantied to work with any random matrix.
```


In [9]:
np.random.seed(123)

def objective(x, C):
    A0 = x[:4].reshape((2, 2))  
    B0 = x[4:8].reshape((2, 2))
    A0 = A0 + A0.T
    B0 = B0 + B0.T
    D00 = np.kron(A0, B0)
    return - np.trace(D00 @ C)

C = np.random.rand(4, 4)

A0_initial = np.eye(2)
B0_initial = np.eye(2)
initial_guess = np.hstack([A0_initial.ravel(), B0_initial.ravel()])

In [10]:
def constraint(x, sign=+1.):
    A0 = x.reshape((2, 2))
    A0 = A0 + A0.T
    U, s, Vt = linalg.svd(np.eye(2) + sign * A0)
    return np.min(np.sign(s)) * np.prod(np.abs(s))

constraints = [
    {'type': 'ineq', 'fun': lambda x: constraint(x[:4], sign=-1.)},
    {'type': 'ineq', 'fun': lambda x: constraint(x[:4], sign=+1.)},
    {'type': 'ineq', 'fun': lambda x: constraint(x[4:8], sign=-1.)},
    {'type': 'ineq', 'fun': lambda x: constraint(x[4:8], sign=+1.)},
]

result = optimize.minimize(objective, initial_guess, args=(C,), constraints=constraints)
print(result)

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -1.0327977281822147e+17
       x: [ 6.132e+07  5.258e+07  5.258e+07  6.376e+07  4.866e+07
            5.773e+07  5.773e+07  5.337e+07]
     nit: 5
     jac: [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
            0.000e+00  0.000e+00  0.000e+00]
    nfev: 45
    njev: 5


In [11]:
c1 = constraint(result.x[:4], sign=-1.)  # 4583471366310759.0
c2 = constraint(result.x[:4], sign=+1.)  # 4583471866661250.0
c3 = constraint(result.x[4:8], sign=-1.) # 2940141030255427.5
c4 = constraint(result.x[4:8], sign=+1.) # 2940140622105013.0

In [12]:
print(c1, c2, c3, c4)

250175242.78364667 250175246.7836467 231100091.64182442 231100091.64182442
